In [1]:
import numpy as np 
import pandas as pd 

data = pd.read_csv('data.csv')

test_set = data[data['shot_made_flag'].isnull()]
training_set = data.dropna()
test_set.to_csv('test_set.csv')
training_set.to_csv('training_set.csv')

In [2]:
training_data = pd.read_csv("training_set.csv")
training_data = pd.DataFrame(training_data)
training_data = training_data.drop(training_data.columns[0], axis = 1)
training_data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
1,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
2,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
3,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5
4,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,9,3,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,6


In [3]:
irrelevant = ["lon", "lat", "team_id", "team_name", "game_id", "game_event_id", "game_date"]
training_data = training_data.drop(columns=irrelevant)
training_data

,action_type,combined_shot_type,loc_x,loc_y,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,-157,0,10,1,0,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,LAL @ POR,POR,2
1,Jump Shot,Jump Shot,-101,135,7,1,0,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,LAL @ POR,POR,3
2,Jump Shot,Jump Shot,138,175,6,1,0,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,LAL @ POR,POR,4
3,Driving Dunk Shot,Dunk,0,0,6,2,0,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,LAL @ POR,POR,5
4,Jump Shot,Jump Shot,-145,-11,9,3,0,2000-01,32,14,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,LAL @ POR,POR,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25692,Driving Layup Shot,Layup,0,0,7,4,1,1999-00,4,0,0.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,LAL vs. IND,IND,30692
25693,Jump Shot,Jump Shot,1,48,6,4,1,1999-00,5,4,0.0,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,LAL vs. IND,IND,30693
25694,Running Jump Shot,Jump Shot,-134,166,3,4,1,1999-00,28,21,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,LAL vs. IND,IND,30695
25695,Jump Shot,Jump Shot,31,267,2,4,1,1999-00,10,26,0.0,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,LAL vs. IND,IND,30696


In [4]:
tr = training_data.drop("shot_made_flag", axis = 1)

In [5]:
tr_labels = training_data["shot_made_flag"].copy()

In [6]:
tr_labels

0        0.0
1        1.0
2        0.0
3        1.0
4        0.0
        ... 
25692    0.0
25693    0.0
25694    1.0
25695    0.0
25696    0.0
Name: shot_made_flag, Length: 25697, dtype: float64

In [7]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25697 entries, 0 to 25696
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   action_type         25697 non-null  object
 1   combined_shot_type  25697 non-null  object
 2   loc_x               25697 non-null  int64 
 3   loc_y               25697 non-null  int64 
 4   minutes_remaining   25697 non-null  int64 
 5   period              25697 non-null  int64 
 6   playoffs            25697 non-null  int64 
 7   season              25697 non-null  object
 8   seconds_remaining   25697 non-null  int64 
 9   shot_distance       25697 non-null  int64 
 10  shot_type           25697 non-null  object
 11  shot_zone_area      25697 non-null  object
 12  shot_zone_basic     25697 non-null  object
 13  shot_zone_range     25697 non-null  object
 14  matchup             25697 non-null  object
 15  opponent            25697 non-null  object
 16  shot_id             25

In [8]:
tr_num = tr.select_dtypes(include='int64')

In [9]:
tr_num

,loc_x,loc_y,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_id
0,-157,0,10,1,0,22,15,2
1,-101,135,7,1,0,45,16,3
2,138,175,6,1,0,52,22,4
3,0,0,6,2,0,19,0,5
4,-145,-11,9,3,0,32,14,6
...,...,...,...,...,...,...,...,...
25692,0,0,7,4,1,4,0,30692
25693,1,48,6,4,1,5,4,30693
25694,-134,166,3,4,1,28,21,30695
25695,31,267,2,4,1,10,26,30696


In [10]:
tr_cat = tr.select_dtypes(include='object')

In [11]:
tr_cat

,action_type,combined_shot_type,season,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,matchup,opponent
0,Jump Shot,Jump Shot,2000-01,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,LAL @ POR,POR
1,Jump Shot,Jump Shot,2000-01,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,LAL @ POR,POR
2,Jump Shot,Jump Shot,2000-01,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,LAL @ POR,POR
3,Driving Dunk Shot,Dunk,2000-01,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,LAL @ POR,POR
4,Jump Shot,Jump Shot,2000-01,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,LAL @ POR,POR
...,...,...,...,...,...,...,...,...,...
25692,Driving Layup Shot,Layup,1999-00,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,LAL vs. IND,IND
25693,Jump Shot,Jump Shot,1999-00,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,LAL vs. IND,IND
25694,Running Jump Shot,Jump Shot,1999-00,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,LAL vs. IND,IND
25695,Jump Shot,Jump Shot,1999-00,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,LAL vs. IND,IND


In [12]:
from sklearn.preprocessing import LabelBinarizer
lbe = LabelBinarizer()

In [15]:
tr_action = tr_cat["action_type"]
tr_combinedshot = tr_cat["combined_shot_type"]
tr_season = tr_cat["season"]
tr_shottype = tr_cat["shot_type"]
tr_shotarea = tr_cat["shot_zone_area"]
tr_shotbasic = tr_cat["shot_zone_basic"]
tr_shotrange = tr_cat["shot_zone_range"]
tr_matchup = tr_cat["matchup"]
tr_opponent = tr_cat["opponent"]

In [16]:
tr_action_1hot = lbe.fit_transform(tr_action)
tr_combinedshot_1hot = lbe.fit_transform(tr_combinedshot)
tr_season_1hot = lbe.fit_transform(tr_season)
tr_shottype_1hot = lbe.fit_transform(tr_shottype)
tr_shotarea_1hot = lbe.fit_transform(tr_shotarea)
tr_shotbasic_1hot = lbe.fit_transform(tr_shotbasic)
tr_shotrange_1hot = lbe.fit_transform(tr_shotrange)
tr_matchup_1hot = lbe.fit_transform(tr_matchup)
tr_opponent_1hot = lbe.fit_transform(tr_opponent)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin): 
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names 
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [19]:
from sklearn.pipeline import FeatureUnion

num_attribs = list(tr_num)
cat_attribs = list(tr_cat)

In [20]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('std_scaler', StandardScaler()),
])

In [21]:
cat_pipeline = Pipeline([
             ('selector', DataFrameSelector(cat_attribs)),
             ('one_hot_encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')),
])

In [22]:
full_pipeline = FeatureUnion(transformer_list=[
             ("num_pipeline", num_pipeline),
             ("cat_pipeline", cat_pipeline),
])

In [23]:
tr_prepared = full_pipeline.fit_transform(tr)

In [24]:
tr_prepared

array([[-1.49129575, -1.03524606,  1.48105426, ...,  0.        ,
         0.        ,  0.        ],
       [-0.98253325,  0.49622757,  0.6120956 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18879244,  0.94999754,  0.32244271, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.28233972,  0.8478993 , -0.54651594, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.21669265,  1.99366846, -0.83616883, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05585869, -0.21846012, -1.4154746 , ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score

In [26]:
tr_prepared.shape

(25697, 216)

In [27]:
lin_reg = LinearRegression()
lin_reg.fit(tr_prepared, tr_labels)

some_data = tr.iloc[:5]
some_labels = tr_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:\t", lin_reg.predict(some_data_prepared))

print("Labels:\t\t", list(some_labels))

tr_predictions = lin_reg.predict(tr_prepared)
lin_mse = mean_squared_error(tr_labels, tr_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Predictions:	 [0.35839844 0.40380859 0.4296875  1.01074219 0.33886719]
Labels:		 [0.0, 1.0, 0.0, 1.0, 0.0]


0.4571909659822704

In [28]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(tr_prepared, tr_labels)

some_data = tr.iloc[:5]
some_labels = tr_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:\t", tree_reg.predict(some_data_prepared))

print("Labels:\t\t", list(some_labels))

tr_predictions = tree_reg.predict(tr_prepared)
tree_mse = mean_squared_error(tr_labels, tr_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

Predictions:	 [0. 1. 0. 1. 0.]
Labels:		 [0.0, 1.0, 0.0, 1.0, 0.0]


0.0

In [29]:
def display_scores(scores):
    print("Scores:", scores)
    print("Scores Means:", scores.mean())
    print("Standard Deviation:", scores.std())

In [30]:
scores = cross_val_score(tree_reg, tr_prepared, tr_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

Scores: [0.63735836 0.63796857 0.64101087 0.64313193 0.62782436 0.65867453
 0.64554746 0.64295445 0.63656582 0.67600825]
Scores Means: 0.6447044591839097
Standard Deviation: 0.012827671848149116


In [31]:
lin_scores = cross_val_score(lin_reg, tr_prepared, tr_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [4.62252405e-01 4.55569917e-01 1.54209782e+12 2.92604346e+10
 4.57886841e-01 4.67942061e-01 8.41139027e+11 3.81657987e+11
 4.61483256e-01 4.80903197e-01]
Scores Means: 279415527027.6829
Standard Deviation: 495113035385.4382


In [32]:
forest_reg = RandomForestRegressor()
forest_reg.fit(tr_prepared, tr_labels)

some_data = tr.iloc[:5]
some_labels = tr_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:\t", forest_reg.predict(some_data_prepared))

print("Labels:\t\t", list(some_labels))

tr_predictions = forest_reg.predict(tr_prepared)
forest_mse = mean_squared_error(tr_labels, tr_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

Predictions:	 [0.09 0.83 0.29 0.85 0.05]
Labels:		 [0.0, 1.0, 0.0, 1.0, 0.0]


0.1743779966683547

In [33]:
forest_scores = cross_val_score(forest_reg, tr_prepared, tr_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [0.46758869 0.46174904 0.46398305 0.46401903 0.46024871 0.46870299
 0.46558494 0.46815736 0.46742839 0.48084257]
Scores Means: 0.46683047598232985
Standard Deviation: 0.005378123367646603


In [34]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2,3,4]}
]
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error")

grid_search.fit(tr_prepared, tr_labels)

print("Best Params:\t", grid_search.best_params_)
print("Best Estimator:\t", grid_search.best_estimator_)

Best Params:	 {'max_features': 6, 'n_estimators': 30}
Best Estimator:	 RandomForestRegressor(max_features=6, n_estimators=30)


In [35]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.5442502806853035 {'max_features': 2, 'n_estimators': 3}
0.4879463778632904 {'max_features': 2, 'n_estimators': 10}
0.47364003724611103 {'max_features': 2, 'n_estimators': 30}
0.5350764791147531 {'max_features': 4, 'n_estimators': 3}
0.48589365607442875 {'max_features': 4, 'n_estimators': 10}
0.47145188115058956 {'max_features': 4, 'n_estimators': 30}
0.5336159744853651 {'max_features': 6, 'n_estimators': 3}
0.4841083488119721 {'max_features': 6, 'n_estimators': 10}
0.4702878037604735 {'max_features': 6, 'n_estimators': 30}
0.5339480328371116 {'max_features': 8, 'n_estimators': 3}
0.48640036252645374 {'max_features': 8, 'n_estimators': 10}
0.4710722126783027 {'max_features': 8, 'n_estimators': 30}
0.5417420605352811 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.4929917609846643 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.5344054636563095 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.4904683641194759 {'bootstrap': False, 'max_features': 3

In [36]:
test_data = pd.read_csv("test_set.csv")

In [37]:
test_data = test_data.drop(test_data.columns[0], axis = 1)

In [38]:
test_data_id = test_data["shot_id"]
test_data_id = pd.DataFrame(test_data_id)

In [39]:
x_test = test_data.drop("shot_made_flag", axis = 1)

In [40]:
x_test

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,254,20000012,34.0163,1,28,-118.2688,8,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,8
2,Driving Layup Shot,Layup,100,20000019,34.0443,0,0,-118.2698,0,1,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,17
3,Driving Layup Shot,Layup,249,20000019,34.0443,0,0,-118.2698,10,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,20
4,Jump Shot,Jump Shot,4,20000047,33.9683,163,76,-118.1068,11,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Jump Shot,Jump Shot,386,49900087,33.8223,-23,222,-118.2928,7,4,...,2PT Field Goal,Center(C),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-06-16,LAL @ IND,IND,30669
4996,Tip Shot,Tip Shot,213,49900088,34.0443,0,0,-118.2698,0,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30681
4997,Running Jump Shot,Jump Shot,226,49900088,33.9963,-68,48,-118.3378,11,3,...,2PT Field Goal,Left Side(L),In The Paint (Non-RA),8-16 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30683
4998,Running Jump Shot,Jump Shot,268,49900088,33.9513,16,93,-118.2538,5,3,...,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30687


In [41]:
final_model = grid_search.best_estimator_

In [42]:
x_test_prepared = full_pipeline.transform(x_test)

In [43]:
final_predictions = final_model.predict(x_test_prepared)

In [44]:
final_predictions

array([0.6       , 0.4       , 0.56666667, ..., 0.43333333, 0.76666667,
       0.56666667])

In [45]:
final_predictions = pd.DataFrame(final_predictions)

In [46]:
final_predictions

,0
0,0.600000
1,0.400000
2,0.566667
3,0.566667
4,0.400000
...,...
4995,0.400000
4996,0.433333
4997,0.433333
4998,0.766667


In [47]:
test_data_id

,shot_id
0,1
1,8
2,17
3,20
4,33
...,...
4995,30669
4996,30681
4997,30683
4998,30687


In [48]:
submission_file = pd.concat([test_data_id, final_predictions], axis=1)

In [49]:
submission_file

,shot_id,0
0,1,0.600000
1,8,0.400000
2,17,0.566667
3,20,0.566667
4,33,0.400000
...,...,...
4995,30669,0.400000
4996,30681,0.433333
4997,30683,0.433333
4998,30687,0.766667


In [50]:
submission_file.columns = ["shot_id", "shot_made_flag"]

In [51]:
submission_file

,shot_id,shot_made_flag
0,1,0.600000
1,8,0.400000
2,17,0.566667
3,20,0.566667
4,33,0.400000
...,...,...
4995,30669,0.400000
4996,30681,0.433333
4997,30683,0.433333
4998,30687,0.766667


In [52]:
submission_file.to_csv('submission_file_removedirrelevant.csv')